# FCC Hard Carbon Optimizer - Quick Start

This notebook demonstrates the key features of the FCC Hard Carbon Optimizer.

In [ ]:
import sys
sys.path.insert(0, '../src')

from fcc_hard_carbon import (
    Feedstock, ProcessConditions, HardCarbonPredictor, GoldilocksAnalyzer,
    ProcessOptimizer, SensitivityAnalyzer, EconomicsCalculator
)

## 1. Property Prediction

Predict hard carbon properties from feedstock and process conditions.

In [ ]:
# Define feedstock (FCC decant oil)
feed = Feedstock(
    sulfur_wt_pct=3.5,
    oxygen_wt_pct=1.0,
    aromatics_pct=85.0,
    mcr_wt_pct=22.0
)

# Define process conditions
proc = ProcessConditions(
    temp_C=1100,
    rate_C_min=5.0,
    time_hr=2.0
)

# Predict properties
predictor = HardCarbonPredictor()
result = predictor.predict(feed, proc)

print(f"d002: {result.d002_nm:.4f} nm")
print(f"In Goldilocks (0.37-0.40): {result.in_goldilocks}")
print(f"Capacity: {result.capacity_mAh_g:.0f} mAh/g")
print(f"ICE: {result.ice_pct:.1f}%")
print(f"Quality Score: {result.quality_score}/100")
print(f"Grade: {result.grade}")

## 2. Process Optimization

Use Latin Hypercube Sampling to find optimal conditions.

In [ ]:
optimizer = ProcessOptimizer(feed)
top_conditions = optimizer.optimize(n_samples=500, top_n=5)
stats = optimizer.get_stats()

print(f"Samples in Goldilocks: {stats['goldilocks']}/{stats['total']} ({stats['rate_pct']:.1f}%)")
print("\nTop 5 Conditions:")
for i, r in enumerate(top_conditions, 1):
    print(f"{i}. T={r.temp_C:.0f}°C, Rate={r.rate_C_min:.1f}°C/min, d002={r.d002_nm:.4f}nm, Score={r.score:.1f}")

## 3. Sensitivity Analysis

Identify which parameters have the largest impact on d002.

In [ ]:
sens = SensitivityAnalyzer(feed, proc)
sensitivity = sens.analyze(pct=20)

print(f"Base d002: {sens.base_d002:.4f} nm\n")
print("Parameter Sensitivity (sorted by impact):")
for r in sensitivity:
    print(f"  {r['param']:<15}: Impact = {r['impact']*1000:.2f} mÅ")

## 4. Business Case

Calculate CAPEX, OPEX, NPV, and IRR.

In [ ]:
econ = EconomicsCalculator(fcc_tpy=10000, yield_pct=35)

capex = econ.get_capex()
opex = econ.get_opex()
rev = econ.get_revenue()
fin = econ.get_npv_irr()

print(f"Scale: {10000} TPY FCC Oil → {rev['hc_tpy']:.0f} TPY Hard Carbon")
print(f"\nCAPEX: ₹{capex['total_cr']:.1f} Crore")
print(f"OPEX: ₹{opex['total_cr']:.2f} Crore/year")
print(f"Revenue: ₹{rev['revenue_cr']:.2f} Crore/year")
print(f"\nNPV: ₹{fin['npv_cr']:.1f} Crore")
print(f"IRR: {fin['irr_pct']:.1f}%")
print(f"Payback: {fin['payback_yr']} years")

## 5. Price Scenarios

In [ ]:
import pandas as pd

scenarios = econ.scenarios()
df = pd.DataFrame(scenarios)
df.columns = ['Price Change', 'Price (₹/kg)', 'NPV (Cr)', 'IRR (%)', 'Payback (yr)']
df